### Project 03: Programming in a group - Solution

This notebook presents *one* solution for Project 03. Your solution may be different. If it works and follows OOP concepts, then you should get full credit. This solution uses three classes: MainWindow, NoteForm, and MakeNote to accomplish the desired functionality. It saves the notebook as a json file. You may use a different format. The MakeNote and NoteForm classes are adapted to include the Edit and Snippet functionality.

There are sections of the code that are clunky and that could be more object oriented. This is intentional to give you opportunities to improve the code in project 04.

### Scott Dempwolf, April 30, 2024

In Project 03 we will extend your notebook project. However, you will be working collaboratively in teams of 3 - 5 people and getting the feel of collaboration tools like github to develop code as a team. This is where the OOP concept of encapsulation becomes especially useful. Each person on the team can be responsible for different aspects of the project, say, different classes, with a shared understanding of how they come together.

Goals for Project 03 include:

1. Set up github for sharing and working together
2. Share project 2 code and come up with one shared version
3. Share project 2 content and create shared content file. Add authors to data or metadata
4. Make notes editable after creation. Track edit history
5. Add a new snippet class and functionality to create, save, read, display, and edit code snippets. *you may also achieve this by modifying existing classes*
6. Add snippet content

#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [1]:
# imports
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import datetime # one module for working with dates and times
import json #this solution saves and opens json files. You may use a different file type and change the import accordingly

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x800") # set the default window size
        self.title('Notebook') #set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.notes = [] # list of note objects
        self.note = {}

        # Styling
        self.style = ttk.Style()
        self.style.configure('TFrame', background='gray')
        self.style.configure('TButton', background='blue', font=('Helvetica', 10))
        self.style.configure('TLabel', background='gray', font=('Helvetica', 10))

        self.frame_main = ttk.Frame(self)
        self.frame_main.pack(fill=tk.BOTH, expand=True)

        # Scrollable frame for notes
        self.canvas = tk.Canvas(self.frame_main)
        self.scrollbar = ttk.Scrollbar(self.frame_main, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.frame_notes = self.scrollable_frame
        self.canvas.grid(row=1, column=3, rowspan=6, sticky='nsew')
        self.scrollbar.grid(row=1, column=4, rowspan=6, sticky='ns')

        # Buttons
        btn1 = ttk.Button(self.frame_main, text='Create New Note', command=self.new_note)
        btn1.grid(padx=10, pady=10, row=1, column=1)

        btn2 = ttk.Button(self.frame_main, text='Open Notebook', command=self.open_notebook)
        btn2.grid(padx=10, pady=10, row=2, column=1)

        btn3 = ttk.Button(self.frame_main, text='Save Notebook\nand Refresh', command=self.save_notebook)
        btn3.grid(padx=10, pady=10, row=3, column=1)

        btn4 = ttk.Button(self.frame_main, text='Quit', command=self.destroy)
        btn4.grid(padx=10, pady=10, row=4, column=1)
     
       
    def new_note(self): # opens a new note form
        note_form = NoteForm(self, self.notebook, self.notes)
        return None

    def clear_frame(self, target_frame): # method for clearing old content from the frame
        for widgets in target_frame.winfo_children():
            widgets.destroy()

    def show_notes(self): # generates note objects and displays them in the main window
        self.clear_frame(self.frame_notes) # clears any previous display
        self.notes = [] # resets the notes object list
        for note in self.notebook: # create new note objects from the notebook and display them
            new_note = MakeNote(master=self.frame_notes, note_dict=note, notebook = self.notebook)
            self.notes.append(new_note)
            new_note.pack(padx=10, pady=10)
            new_note.config(height= 4, width=60, wraplength=300, justify=tk.LEFT)
        return None
    
    def open_notebook(self):
        # this opens json files. You may use different file types.
        filepath = filedialog.askopenfilename(initialdir="C:\\Users\\sdemp\\Documents\\GitHub\\Courses\\INST326\\test_files",
                                         filetypes=[("json files", "*.json"), 
                                                    ("csv files", ".csv"),
                                         ("all files", "*.*")])
        file = open(filepath, "r")
        self.notebook = json.load(file) # load the json file into self.notebook as a list of dictionaries
        file.close()
       
        self.show_notes() # once the file is loaded, call the method to display the notes in the window
        return None
    
    def save_notebook(self, notebook): #modifications to accomodate MakeNote.save_close() method
        self.notebook = notebook
        print('save notebook')
        print(self.notebook)
        # the following code saves the notebook as a json file. You  may use different file types
        file = filedialog.asksaveasfile(initialdir="C:\\Users\\sdemp\\Documents\\GitHub\\Courses\\INST326\\test_files",
                                              defaultextension=".json", 
                                        title="notebook01",
                                              filetypes=[("json file", ".json"),
                                             ("all files", ".*")])
        
        json_out = json.dumps(self.notebook, indent=2)
        file.write(json_out)
        
        self.show_notes() # this refreshes the notes display in the main window
        return None


# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example

# PROJECT 03 MODIFICATIONS
# ADD NOTE_ID, AUTHOR, AND SNIPPET FIELDS TO NoteForm

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        # set the new window's default parameters
        self.geometry("600x700") 
        self.title('New Note')
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')

        #PROJECT 03 EDIT
        # moved the definition of self.notebook above the default note
        
        #define self.notebook as the notebook passed from the main window
        self.notebook = notebook
        #self.notes = notes (not necessary)
        
        # PROJECT 03 EDIT: ADD note_id
        self.last_id = len(self.notebook)
        

        
        #define default dummy text (for development purposes only)
        default_note = {"title":"new note title",
                     "text":"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Etiam sit amet suscipit mi, non porttitor mauris. Aliquam in lorem risus. Proin mauris mauris, varius ac vulputate sed, tempor nec lacus. Morbi sodales turpis in placerat semper. Donec bibendum blandit ante sit amet hendrerit.", 
                    "link":"If there is a link with this note enter it here.",
                    "tags":"enter hashtags here",
                    "id":self.last_id + 1,
                    "author":"Scott Dempwolf",
                    "snippet":"# enter executable code snippet here \nprint('Hello World')",
                    "meta":"metadata added at submission"}
                    
        note = default_note # provided in anticipation of note editing functionality
        
        
        # create some labels and put them in the grid
        # we are using the grid layout. Notice the sticky='e' attribute. 
        # this causes the label to 'stick' to the 'east' side of the grid cell
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')

        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note ID:')
        tag_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Author:')
        tag_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Snippet:')
        tag_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')

        # create our note title entry field
        self.note_title = tk.Entry(self.frame_main, width=80)
        self.note_title.grid(padx=10, pady=10, row=1, column=1, sticky='w')
#         self.note_title.insert(0, note["title"]) # adds default text (useful during development)

        # create our note text field
        self.note_text = tk.Text(self.frame_main, height=10, width=60)
        self.note_text.grid(padx=10, pady=10, row=2, column=1)
        self.note_text.insert('1.0', note["text"]) # adds default text (useful during development)

        # create our note link entry field
        self.note_link = tk.Entry(self.frame_main, width=80)
        self.note_link.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.note_link.insert(0, note["link"]) # adds default text (useful during development)

        # create our note tags entry field
        self.note_tags = tk.Entry(self.frame_main, width=80)
        self.note_tags.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.note_tags.insert(0, note["tags"]) # adds default text (useful during development)

        # create our note id entry field
        self.note_id = tk.Entry(self.frame_main, width=80)
        self.note_id.grid(padx=10, pady=10, row=5, column=1, sticky='w')
        self.note_id.insert(0, note["id"]) # adds default text (useful during development)

        # create our note author entry field
        self.note_author = tk.Entry(self.frame_main, width=80)
        self.note_author.grid(padx=10, pady=10, row=6, column=1, sticky='w')
        self.note_author.insert(0, note["author"]) # adds default text (useful during development)

        # create our note snippet field
        self.snippet = tk.Text(self.frame_main, height=10, width=60)
        self.snippet.grid(padx=10, pady=10, row=7, column=1)
        self.snippet.insert('1.0', note["snippet"]) # adds default text (useful during development)        
        
        # create our note meta field if you want to add edit functionality
#         self.note_meta = tk.Entry(self.frame_main, width=80)
#         self.note_meta.grid(padx=10, pady=10, row=5, column=1, sticky='w')
#         self.note_meta.insert(0, note["meta"]) # adds default text (useful during development)
        

        # note that the parameters for the Entry box and Text box are slightly different.
        # The user can create multiple notes with the same note form. Each time the 'submit'
        # button is pressed, a new note is added to the notebook.

        b1 = tk.Button(self.frame_main, text='submit', command=self.submit)
        b1.grid(padx=10, pady=10, row=9, column=1, sticky='w')

        b5 = tk.Button(self.frame_main, text='close', command=self.destroy)
        b5.grid(padx=10, pady=10, row=9, column=0) 
       

    
    def submit(self):
        # calculate the date and time information for the meta field
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        created = datetime.datetime.now()
        
        # get all the input values and put them into a dictionary along with the metadata
        title = self.note_title.get()
        text = self.note_text.get('1.0', 'end').strip('\n')
        link = self.note_link.get()
        tags = self.note_tags.get()
        note_id = self.note_id.get()
        author = self.note_author.get()
        snippet = self.snippet.get('1.0', 'end').strip('\n')
        meta = f'note_id {note_id} created {created}, {local_tz} by {author}'
        note_dict = {'title':title, 'text':text, 'link':link, 'tags':tags, 'note_id':note_id, 'author':author, 'snippet':snippet, 'meta':meta}
        
        # add the dictionary to the notebook
        self.notebook.append(note_dict)
        self.last_id = self.last_id + 1
        self.note_id.delete(0, tk.END)
        self.note_id.insert(0, self.last_id + 1)
        
        return None

# The MakeNote class creates a new note object.
# The MakeNote class is a subclass of the tk.Button class, 
# which means each note instance has tk.Button functionality

class MakeNote(tk.Button):
    def __init__(self, master=None, note_dict=None, notebook=None): # the arguments on this line
        # are inbound, meaning we pass them when we instantiate the object
        super().__init__(master) # on this line we call the __init__ method of tk.Button and pass
        # the master attribute to it. This gives us all the button attributes and functionality
        
        self.notebook = notebook
        
        # define note attributes
        self.title = note_dict['title']
        self.text = note_dict['text']
        self.link = note_dict['link']
        self.tags = note_dict['tags']
        self.note_id = note_dict['note_id']
        self.author = note_dict['author']
        self.snippet = note_dict['snippet']
        self.meta = note_dict['meta']

        # configure note button; this creates a button with two lines of text
        self.config(bg='light gray', text=f"{self.title}\n{self.meta}")
        
        # Bind mouse events
        self.bind("<Enter>", self.on_hover)
        self.bind("<Leave>", self.on_leave)
        self.bind("<Button-1>", self.note_open)

    def save_close(self):
        self.note_window.destroy()
        main_window.save_notebook(notebook=self.notebook)
       

    def on_hover(self, event): # change the background when the cursor hovers over it
        self.config(bg="lightblue")  

    def on_leave(self, event): # change back when not hovering
        self.config(bg="light gray")  # Restore original color
        
    def note_open(self, event): # on mouse click, open note in new top window
        
        # create a new top window
        self.note_window = tk.Toplevel(main_window, bg="light gray", height=600, width=600)
        self.note_window.title(self.title)
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self.note_window)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create labels in the frame
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        title_content = tk.Label(self.frame_main, bg='light gray', text=self.title, wraplength=400, justify=tk.LEFT)
        title_content.grid(padx=10, pady=10, row=1, column=1, sticky='w')        

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')
        text_content = tk.Label(self.frame_main, bg='light gray', text=self.text, wraplength=400, justify=tk.LEFT)
        text_content.grid(padx=10, pady=10, row=2, column=1, sticky='w')

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')
        link_content = tk.Label(self.frame_main, bg='light gray', text=self.link, wraplength=400, justify=tk.LEFT)
        link_content.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        tag_content = tk.Label(self.frame_main, bg='light gray', text=self.tags, wraplength=400, justify=tk.LEFT)
        tag_content.grid(padx=10, pady=10, row=4, column=1, sticky='w')

        note_id_label = tk.Label(self.frame_main, bg='light gray', text='Note ID:')
        note_id_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        note_id_content = tk.Label(self.frame_main, bg='light gray', text=self.note_id, wraplength=400, justify=tk.LEFT)
        note_id_content.grid(padx=10, pady=10, row=5, column=1, sticky='w')

        author_label = tk.Label(self.frame_main, bg='light gray', text='Note Author:')
        author_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        author_content = tk.Label(self.frame_main, bg='light gray', text=self.author, wraplength=400, justify=tk.LEFT)
        author_content.grid(padx=10, pady=10, row=6, column=1, sticky='w')

        snippet_label = tk.Label(self.frame_main, bg='light gray', text='Snippet:')
        snippet_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')
        snippet_content = tk.Label(self.frame_main, bg='light gray', text=self.snippet, wraplength=400, justify=tk.LEFT)
        snippet_content.grid(padx=10, pady=10, row=7, column=1, sticky='w')
        
        meta_label = tk.Label(self.frame_main, bg='light gray', text='Note Meta:')
        meta_label.grid(padx=10, pady=10, row=8, column=0, sticky='e')
        meta_content = tk.Label(self.frame_main, bg='light gray', text=self.meta, wraplength=400, justify=tk.LEFT)
        meta_content.grid(padx=10, pady=10, row=8, column=1, sticky='w')        

        # create a button to close the note window
        b10 = tk.Button(self.frame_main, text='save and close', command=self.save_close)
        b10.grid(padx=10, pady=10, row=9, column=0)

        # create a button to edit the note
        b11 = tk.Button(self.frame_main, text='edit note', command=self.note_edit)
        b11.grid(padx=10, pady=10, row=9, column=1)

    def save_edits(self):
        # calculate the date and time information for the meta field
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        edited = datetime.datetime.now()
        
        # get all the input values and put them into a dictionary along with the metadata
        self.title = self.title_content.get()
        self.text = self.text_content.get('1.0', 'end').strip('\n')
        self.link = self.link_content.get()
        self.tags = self.tag_content.get()
        self.note_id = self.note_id_content.get()
        self.author = self.author_content.get()
        self.snippet = self.snippet_content.get('1.0', 'end').strip('\n')
        
        # modify the metadata to track edits
        self.meta = self.meta + f', edited {edited}, {local_tz}'
        note_dict = {'title':self.title, 'text':self.text, 'link':self.link, 'tags':self.tags, 'note_id':self.note_id, 'author':self.author, 'snippet':self.snippet, 'meta':self.meta}

        new_notebook = []
        for note in self.notebook:
            if note['note_id'] != self.note_id:
                new_notebook.append(note)
            else:
                new_notebook.append(note_dict)
        self.notebook = new_notebook
        
        # destroy the edit window
        self.edit_window.destroy()
        self.note_window.destroy()
        event = None
        self.note_open(event)
        
        
    def note_edit(self):
        # loads note into an editable window
        # create a new top window
        self.edit_window = tk.Toplevel(main_window, bg="light gray", height=700, width=600)
        self.edit_window.title(self.title)


        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self.edit_window)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create labels in the frame
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        self.title_content = tk.Entry(self.frame_main, width=80)
        self.title_content.grid(padx=10, pady=10, row=1, column=1, sticky='w')
        self.title_content.insert(0, self.title)

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')
        self.text_content = tk.Text(self.frame_main, height=8, width=60)
        self.text_content.grid(padx=10, pady=10, row=2, column=1, sticky='w')
        self.text_content.insert('1.0', self.text)

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')
        self.link_content = tk.Entry(self.frame_main, width=80)
        self.link_content.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.link_content.insert(0, self.link)
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        self.tag_content = tk.Entry(self.frame_main, width=80)
        self.tag_content.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.tag_content.insert(0, self.tags)

        note_id_label = tk.Label(self.frame_main, bg='light gray', text='Note ID:')
        note_id_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        self.note_id_content = tk.Entry(self.frame_main, width=80)
        self.note_id_content.grid(padx=10, pady=10, row=5, column=1, sticky='w')
        self.note_id_content.insert(0, self.note_id)

        author_label = tk.Label(self.frame_main, bg='light gray', text='Note Author:')
        author_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        self.author_content = tk.Entry(self.frame_main, width=80)
        self.author_content.grid(padx=10, pady=10, row=6, column=1, sticky='w')
        self.author_content.insert(0, self.author)

        snippet_label = tk.Label(self.frame_main, bg='light gray', text='Snippet:')
        snippet_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')
        self.snippet_content = tk.Text(self.frame_main, height=8, width=60)
        self.snippet_content.grid(padx=10, pady=10, row=7, column=1, sticky='w')
        self.snippet_content.insert('1.0', self.snippet)
        
        meta_label = tk.Label(self.frame_main, bg='light gray', text='Note Meta:')
        meta_label.grid(padx=10, pady=10, row=8, column=0, sticky='e')
        self.meta_content = tk.Label(self.frame_main, bg='light gray', text=self.meta, wraplength=400, justify=tk.LEFT)
        self.meta_content.grid(padx=10, pady=10, row=8, column=1, sticky='w')        

        # create a button to close the edit window
        b10 = tk.Button(self.frame_main, text='close', command=self.edit_window.destroy)
        b10.grid(padx=10, pady=10, row=9, column=0)

        # create a button to edit the note
        b11 = tk.Button(self.frame_main, text='save edits', command=self.save_edits)
        b11.grid(padx=10, pady=10, row=9, column=1)
        
        
# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

In [36]:
# print your notes here

#### Instructions from project 02

The following cells include original instructions from project 02 as reference

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Adapting and Reusing and a class from Project 01

This solution adapts the Cardbutton class (shown below) from project 01 for use as the MakeNote class in this project. Instantiating the notes as buttons allows us to use the same mouse event functionality that we used in Project 01. We can use that functionality to select our notes and display them in a popup or top level window.

In [ ]:
# creates a custom button that inherits all the features of a tk.Button() and adds card attributes
class Cardbutton(tk.Button):
    def __init__(self, master=None, suit=None, rank=None, value=None): # the arguments on this line
        # are inbound, meaning we pass them when we instantiate the object
        super().__init__(master) # on this line we call the __init__ method of tk.Button and pass
        # the master attribute to it. This gives us all the button attributes and functionality
        self.config(bg = 'green')

        # the rest of the attributes in the main __init__ method are card attributes from our former card class
        self.suit = suit
        self.rank = rank # face rank of the card
        self.value = value # integer value of the card
        self.name = f"{rank}_of_{suit}"
        self.face = f"images\\{rank} of {suit}.png" # relative address
        self.back = "images\\BackOfCard.png"  # relative address
        self.suit_val = suits.index(suit)
        self.facetk = tk.PhotoImage(file = self.face) # calls the tk.PhotoImage method which creates an image object
        self.backtk = tk.PhotoImage(file = self.back)
        self.display = self.facetk # sets the default display
        self.hand = None # an attribute that allows cards to be assigned to specific hands


        # Bind mouse events
        self.bind("<Enter>", self.on_hover)
        self.bind("<Leave>", self.on_leave)
        self.bind("<Button-1>", self.ace_toggle)

    def on_hover(self, event): # change the background and card image when the cursor hovers over it
        self.config(bg="lightblue")  

    def on_leave(self, event): # change back when not hovering
        self.config(bg="green", image=self.display)  # Restore original color